In [216]:
import pandas as pd
import numpy as np
import glob

In [217]:
zo = pd.DataFrame()
for file in glob.glob("dbm*.csv"):
    df = pd.read_csv(file)
    zo = zo.append(df, ignore_index=True)

In [218]:
dbm = pd.DataFrame()
for file in glob.glob("att*.csv"):
    df = pd.read_csv(file, skipinitialspace=True, 
            usecols=["Insertion Order", "Insertion Order ID", "Line Item", "Line Item ID", "Revenue (Adv Currency)"])
    dbm = dbm.append(df, ignore_index=True)

In [219]:
IOIDmap = pd.read_csv("map.csv", usecols=["IOID", "IOName", "Targeting"])

In [220]:
dbm.columns = ["IO", "IOID", "LI", "LIID", "Spend"]

In [221]:
dbm = dbm.drop(dbm.index[dbm.tail(5).index.values])
dbm = dbm.loc[dbm.LI.isnull() == False]

In [222]:
dbm.IOID = dbm.IOID.astype("int64")
dbm.LIID = dbm.LIID.astype("int64")

In [223]:
#pd.DataFrame(dbm.IOID.unique(), columns=["uniqueIOID"]).to_csv(
#    "D:\\Users\\ylin\\Desktop\\Aviva ATT output\\"+"uniqueIOID.csv", index=False)

In [224]:
dbm_ = pd.DataFrame(dbm.pivot_table(values="Spend", index=["IOID", "IO"], aggfunc=np.sum)).reset_index()

In [225]:
zo_ = pd.DataFrame(zo.pivot_table(values="zo_attr", index="dbm_insertion_order_id", aggfunc=np.sum)).reset_index()

In [226]:
zo_.columns = ["IOID", "zo_attr"]

In [227]:
data = dbm_.merge(zo_, on="IOID", how="left")

In [228]:
data = data.merge(IOIDmap[["IOID", "Targeting"]], on="IOID", how="left")

In [229]:
data_ = data.pivot_table(values=["Spend", "zo_attr"], aggfunc=np.sum, index="Targeting").reset_index()

In [230]:
data_["zoCPQ"] = data_.Spend / data_.zo_attr

In [231]:
data_.to_csv("D:\\Users\\ylin\\Desktop\\"+"latest CPQ.csv", index=False)

In [232]:
data_

,Targeting,Spend,zo_attr,zoCPQ
0,Prospecting,128433.341320,15844.730934,8.105745
1,Remarketing existing,17893.703917,1943.679811,9.206097
2,Remarketing non-existing,180101.571548,28978.067045,6.215100


In [233]:
#Output the new IOs from DBM which are not mapped
IOIDmap_inc = list(set(np.unique(dbm.IOID.values)) - set(IOIDmap.IOID.values))
IOIDmap_inc = pd.DataFrame(data=IOIDmap_inc, columns=["IOID"])
IOIDmap = IOIDmap.append(IOIDmap_inc, ignore_index=True)
IOIDmap.to_csv("map.csv", index=False)